In [ ]:
class ToDo:

    progress = 0  # class-level variable

    def __init__(self, description, status):
        self.description = description
        self.is_done = status

    def mark_as_done(self):
        if not self.is_done: # increment progress if needed
            ToDo.progress = ToDo.progress + 1
        self.is_done = True
        
    def mark_as_pending(self):
        if self.is_done: # decrement progress if needed
            ToDo.progress = ToDo.progress - 1
        self.is_done = False
        
class Deadline(ToDo):
    
    progress = 0
    
    def __init__(self, description, status, deadline):
        self.deadline = deadline
        ToDo.__init__(self, description, status)
        
    def mark_as_done(self):
        if not self.is_done: # increment progress if needed
            Deadline.progress = Deadline.progress + 1
        self.is_done = True
        
    def mark_as_pending(self):
        if self.is_done: # decrement progress if needed
            Deadline.progress = Deadline.progress - 1
        self.is_done = False
        
        
items = []
items.append(ToDo("Read book", False))
items.append(ToDo("borrow book", False))
items.append(Deadline("Make coffee", False, "Monday"))

def get_current_progress():
    status = {'Todo': 0, 'Deadline': 0}
    tFlag = 0
    dFlag = 0
    
    for obj in items:
        if isinstance(obj, Deadline) and dFlag == 0:
            print("added to deadline count\n")
            status['Deadline'] = obj.progress
            dFlag = 1
        elif tFlag == 0:
            status['Todo'] = obj.progress
            tFlag = 1
    print(">>> Progress for this session: todos {} deadlines {}".format(status['Todo'], status['Deadline']))

items[0].mark_as_done()
get_current_progress()
items[2].mark_as_done()
get_current_progress()
items[2].mark_as_pending()
get_current_progress()

In [2]:
class ToDo:
    
    progress = 0 # class-level variable
    
    TYPE_KEY = 'T'
    
    def __init__(self, description, status):
        self.description = description
        self.is_done = status

    def mark_as_done(self):
        if not self.is_done: # increment progress if needed
            ToDo.progress = ToDo.progress + 1
        self.is_done = True

    def mark_as_pending(self):
        if self.is_done: # decrement progress if needed
            if ToDo.progress != 0:
                ToDo.progress = ToDo.progress - 1
        self.is_done = False

    def as_string(self):
        """ Return the details of todo object as a formatted string"""
        status = 'X' if self.is_done else '-'
        return status.center(6) + ' ' + '| ' + ' '.center(5) \
    + '| ' + self.description.ljust(27) + ' ' + '| ' + '-'

    def as_csv(self):
        """ Return the details of todo object as a list,
        suitable to be stored in a csv file.
        """
        return ['T', self.description, 'done' if self.is_done else 'pending']
    
    def as_item(self):
        """ Return the name of the item as a string."""
        return self.description
    
class Deadline(ToDo):
    
    progress = 0 # class level variable
    
    TYPE_KEY = 'D'
    
    def __init__(self, description, status, deadline):
        self.deadline = deadline
        super().__init__(self, description, status)
        
    def mark_as_done(self):
        if not self.is_done: # increment progress if needed
            Deadline.progress = Deadline.progress + 1
        self.is_done = True
        
    def mark_as_pending(self):
        if self.is_done: # decrement progress if needed
            if Deadline.progress != 0:
                Deadline.progress = Deadline.progress - 1
        self.is_done = False
    
    def as_csv(self):
        """ Return the details of deadline object as a list,
        suitable to be stored in a csv file.
        """
        return ['D', self.description, 'done' if self.is_done else 'pending', self.deadline]
    
    def as_string(self):
        """ Return the details of deadline object as a string"""
        status = 'X' if self.is_done else '-'
        return status.center(6) + ' ' + '| '  + ' '.center(5) \
    +  '| ' + self.description.ljust(27) + ' ' + '| ' + self.deadline
    
d = Deadline("Return books", "FALSE", "Monday")
print(d)

TypeError: __init__() takes 3 positional arguments but 4 were given

In [23]:
class ToDo:
    
    progress = 0 # class-level variable
    TYPE_KEY = 'T'

    def __init__(self, description, status):
        self.description = description
        self.is_done = status

    def mark_as_done(self):
        if not self.is_done: # increment progress if needed
            ToDo.progress = ToDo.progress + 1
        self.is_done = True

    def mark_as_pending(self):
        self.is_done = False

    def __status_as_icon(self):
        return 'X' if self.is_done else '-'

    def __str__(self):
        return '(' + self.__status_as_icon() + ') ' + self.description

    # more code ...


class Deadline (ToDo):
    
    progress = 0 # class-level variable
    TYPE_KEY = 'D'

    def __init__(self, description, status, by):
        super().__init__(description, status)
        self.by = by

    def __str__(self):
        return super().__str__() + ' [by: ' + self.by + ']'

t = ToDo("Return book", False)
print(t.is_done)
t.mark_as_done()
print(t.is_done)
print(t.progress)

d = Deadline("Return book", "False", "Monday")
# print(d)

False
True
1
